### 교차 검증(Cross Validation)

#### 테스트 데이터에만 과적합 될 수 있으므로 데이터를 여러개로 나누어 테스트를 여러번 수행하여 평균 정확도를 구함

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
iris_data = load_iris()

X_train,X_test,y_train,y_test = train_test_split(iris_data.data,iris_data.target,
                                                test_size=0.2,
                                                random_state=11)

dt_clf = DecisionTreeClassifier(random_state=11)

# 학습
dt_clf.fit(X_train,y_train)

# 예측
pred = dt_clf.predict(X_test)


# 정확도
print('정확도:{0:.4f}'.format(accuracy_score(y_test,pred))) # 정확도:0.9333

# 교차 검증(cross validation)
cv_score = cross_val_score(dt_clf,iris_data.data,iris_data.target,
               scoring='accuracy',cv=3)
# 데이터 셋을 3개로 쪼개어 검증(fit.predict를 3회 수행)
# 내부적으로 StratifedKFold가 사용됨, 평가 지표를 하나만 구할 수 있어서
# StratifedKFold 사용 권장

print('교차 검증 정확도:',cv_score)
print('교차 검증 평균 정확도:',cv_score.mean())

정확도:0.9333
교차 검증 정확도: [0.98 0.92 0.98]
교차 검증 평균 정확도: 0.96


### K-Fold 교차 검증
#### K번 만큼 폴드(Fold)된 각각의 데이터 세트로 학습과 검증을 K번 만큼 반복하여 평균 정확도를 구함

In [21]:
from sklearn.model_selection import KFold
iris = load_iris()
features = iris.data   # X, 피처
label = iris.target    # Y, 레이블

features.shape  # (150, 4)
label.shape     # (150,)

dt_clf = DecisionTreeClassifier(random_state=11)  # Estimator 생성

(150,)

In [31]:
# 5개 폴드 세트로 분리
Kfold = KFold(n_splits=5)
cv_accuracy = []
n_iter = 0

for train_index,test_index in Kfold.split(features) : 
# Kflod.split()함수 : X 데이터 features를 분리하여 학습/검증용 데이터의 인덱스를 반환

# 150개 데이터
# k값(n_splits) : 폴드 세트 갯수, 학습 : 검증   검증 데이터의 비율
    
    X_train = features[train_index]
    X_test = features[test_index]
    
    y_train = label[train_index]
    y_test = label[test_index]
    
    # 학습 및 예측
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter += 1
    
    # 반복 하여 정확도 측정
    accuracy = round(accuracy_score(y_test,pred),4)
    cv_accuracy.append(accuracy)
    
    
    print('#',n_iter,'번 폴드 교차 검증 정확도:',accuracy,
         '학습 데이터의 크기:',X_train.shape[0],
         '검증 데이터의 크기:',X_test.shape[0])
    print('학습 데이터의 인덱스:\n',train_index)
    print('검증 데이터의 인덱스:\n',test_index)
    
print('전체 폴드의 정확도:',cv_accuracy)

# KFold로 반복된 정확도를 합하여 평균 정확도를 계산
print('K-Fold 교차 검증 평균 정확도:',round(np.mean(cv_accuracy),4)) # 0.92

# 1 번 폴드 교차 검증 정확도: 1.0 학습 데이터의 크기: 120 검증 데이터의 크기: 30
학습 데이터의 인덱스:
 [ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
검증 데이터의 인덱스:
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
# 2 번 폴드 교차 검증 정확도: 0.9667 학습 데이터의 크기: 120 검증 데이터의 크기: 30
학습 데이터의 인덱스:
 [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94 

### Stratified K-Fold : 필수 사용
#### label 이 지나치게 불균형을 분포를 이룰 때 레이블의 분포를 균일하게 폴드시키는 방식
#### 분류 모델에서만 가능(회귀 모델은 지원되지 않음)

In [36]:
# kFold 에서 shuffle 사용 : 레이블의 분포가 균형을 이루지 않는다
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data,columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

kfold = KFold(n_splits=5,shuffle=True) # shuffle과 Stratfied와는 성능이 다름
n_iter = 0 

for train_index,test_index in kfold.split(features):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('\n[',n_iter,'번 폴드 세트]')
    print(label_train.value_counts()) # 120개
    print(label_test.value_counts())  # 30개 , 레이블이 불균형 분포를 이룬다


[ 1 번 폴드 세트]
0    41
1    40
2    39
Name: label, dtype: int64
2    11
1    10
0     9
Name: label, dtype: int64

[ 2 번 폴드 세트]
1    45
0    42
2    33
Name: label, dtype: int64
2    17
0     8
1     5
Name: label, dtype: int64

[ 3 번 폴드 세트]
2    46
0    43
1    31
Name: label, dtype: int64
1    19
0     7
2     4
Name: label, dtype: int64

[ 4 번 폴드 세트]
1    46
2    41
0    33
Name: label, dtype: int64
0    17
2     9
1     4
Name: label, dtype: int64

[ 5 번 폴드 세트]
2    41
0    41
1    38
Name: label, dtype: int64
1    12
2     9
0     9
Name: label, dtype: int64


In [38]:
# StratifiedKFold : 레이블의 분포가 균일하게 분리된다, 정확도 향상
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
n_iter = 0

for train_index, test_index in skf.split(iris_df,iris_df['label']):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('\n[',n_iter,'번 폴드 세트]')
    print(label_train.value_counts()) # 120개
    print(label_test.value_counts())  # 30개 , 
    # 원본과 같은 비율로 레이블이 균형 분포를 이룬다


[ 1 번 폴드 세트]
2    40
1    40
0    40
Name: label, dtype: int64
2    10
1    10
0    10
Name: label, dtype: int64

[ 2 번 폴드 세트]
2    40
1    40
0    40
Name: label, dtype: int64
2    10
1    10
0    10
Name: label, dtype: int64

[ 3 번 폴드 세트]
2    40
1    40
0    40
Name: label, dtype: int64
2    10
1    10
0    10
Name: label, dtype: int64

[ 4 번 폴드 세트]
2    40
1    40
0    40
Name: label, dtype: int64
2    10
1    10
0    10
Name: label, dtype: int64

[ 5 번 폴드 세트]
2    40
1    40
0    40
Name: label, dtype: int64
2    10
1    10
0    10
Name: label, dtype: int64


In [42]:
# StratifiedKFold를 사용하여 학습 및 예측과 정확도 측정(회귀 모델에서는 사용불가능)
skf = StratifiedKFold(n_splits=5)
cv_accuracy = []
n_iter = 0

for train_index,test_index in skf.split(iris_df,iris_df['label']): # Y레이블을 반드시 인자로 사용
    X_train = features[train_index]
    X_test = features[test_index]
    
    y_train = label[train_index]
    y_test = label[test_index]
    
    # 학습 및 예측
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter += 1
    
    # 반복 하여 정확도 측정
    accuracy = round(accuracy_score(y_test,pred),4)
    cv_accuracy.append(accuracy)
    
    
    print('#',n_iter,'번 폴드 교차 검증 정확도:',accuracy,
         '학습 데이터의 크기:',X_train.shape[0],
         '검증 데이터의 크기:',X_test.shape[0])
    print('학습 데이터의 인덱스:\n',train_index)
    print('검증 데이터의 인덱스:\n',test_index)
    
print('전체 폴드의 정확도:',cv_accuracy)

# StratifiedKFold로 반복된 정확도를 합하여 평균 정확도를 계산
print('StratifiedKFold 교차 검증 평균 정확도:',round(np.mean(cv_accuracy),4))

# StratifiedKFold로 레이블의 분포를 원본과 같이 균일하게 폴드를 생성하여 학습시키므로
# 정확도가 향상됨 : 0.92 ----> 0.96

# 1 번 폴드 교차 검증 정확도: 0.9667 학습 데이터의 크기: 120 검증 데이터의 크기: 30
학습 데이터의 인덱스:
 [ 10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
검증 데이터의 인덱스:
 [  0   1   2   3   4   5   6   7   8   9  50  51  52  53  54  55  56  57
  58  59 100 101 102 103 104 105 106 107 108 109]
# 2 번 폴드 교차 검증 정확도: 0.9667 학습 데이터의 크기: 120 검증 데이터의 크기: 30
학습 데이터의 인덱스:
 [  0   1   2   3   4   5   6   7   8   9  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  54  55  56  57  58  59  70  71  72  73
  74  75  76

### GridSearchCV
#### 교차 검증과 파라메터 집합을 만들어 주면 최적의 파라메터 값을 구해줌

In [43]:
from sklearn.model_selection import GridSearchCV

X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,
                                                test_size=0.2,
                                                random_state=11)
dt_clf = DecisionTreeClassifier(random_state=11) # estimator

# 파라메터들을 dict형태로 설정
parameters = {'max_depth':[1,2,3],'min_samples_split':[2,3]}

In [51]:
grid_tree = GridSearchCV(dt_clf,parameters,refit=True,
                        return_train_score=True)
grid_tree.fit(X_train,y_train)
scores_df = pd.DataFrame(grid_tree.cv_results_)
scores_df[['params','mean_test_score']]


,params,mean_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.675000
1,"{'max_depth': 1, 'min_samples_split': 3}",0.675000
2,"{'max_depth': 2, 'min_samples_split': 2}",0.950000
3,"{'max_depth': 2, 'min_samples_split': 3}",0.950000
4,"{'max_depth': 3, 'min_samples_split': 2}",0.958333
5,"{'max_depth': 3, 'min_samples_split': 3}",0.958333


In [55]:
print('GridSearchCV 최적 파라메터:',grid_tree.best_params_)
print('GridSearchCV 최적 파라메터:',grid_tree.best_score_)

GridSearchCV 최적 파라메터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최적 파라메터: 0.9583333333333333


In [58]:
pred = grid_tree.predict(X_test)
print('정확도:',accuracy_score(y_test,pred))

정확도: 0.9333333333333333


In [63]:
estimator = grid_tree.best_estimator_
print(estimator)
pred = estimator.predict(X_test)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=11, splitter='best')
